<a href="https://colab.research.google.com/github/RenChoi/pickme/blob/main/%D0%BF%D0%B8%D1%82%D0%BE%D0%BD%D1%87%D0%B8%D0%BA_2204.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Мазанов Артем 09-313

1. Создание базы данных и таблиц

In [1]:
import sqlite3

conn = sqlite3.connect('library.db')
cursor = conn.cursor()

cursor.execute('''
CREATE TABLE IF NOT EXISTS books (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT NOT NULL,
    author TEXT NOT NULL,
    year_published INTEGER,
    genre TEXT
)
''')

conn.commit()

2. Вставка данных

In [2]:

books = [
    ("The Great Gatsby", "F. Scott Fitzgerald", 1925, "Fiction"),
    ("1984", "George Orwell", 1949, "Dystopian"),
    ("To Kill a Mockingbird", "Harper Lee", 1960, "Classic")
]

cursor.executemany('''
INSERT INTO books (title, author, year_published, genre)
VALUES (?, ?, ?, ?)
''', books)

conn.commit()

3. Выборка всех данных

In [3]:
cursor.execute("SELECT * FROM books")
all_books = cursor.fetchall()
print("Все книги:")
for book in all_books:
    print(book)

Все книги:
(1, 'The Great Gatsby', 'F. Scott Fitzgerald', 1925, 'Fiction')
(2, '1984', 'George Orwell', 1949, 'Dystopian')
(3, 'To Kill a Mockingbird', 'Harper Lee', 1960, 'Classic')


4. Фильтрация данных

In [4]:
cursor.execute("SELECT * FROM books WHERE year_published > 1950")
books_after_1950 = cursor.fetchall()
print("\nКниги после 1950 года:")
for book in books_after_1950:
    print(book)


Книги после 1950 года:
(3, 'To Kill a Mockingbird', 'Harper Lee', 1960, 'Classic')


5. Использование оператора LIKE

In [5]:
cursor.execute("SELECT * FROM books WHERE title LIKE 'T%'")
books_starting_with_t = cursor.fetchall()
print("\nКниги, название которых начинается на 'T':")
for book in books_starting_with_t:
    print(book)


Книги, название которых начинается на 'T':
(1, 'The Great Gatsby', 'F. Scott Fitzgerald', 1925, 'Fiction')
(3, 'To Kill a Mockingbird', 'Harper Lee', 1960, 'Classic')


6. Сортировка данных

In [6]:
cursor.execute("SELECT * FROM books ORDER BY year_published ASC")
sorted_books = cursor.fetchall()
print("\nКниги, отсортированные по году публикации:")
for book in sorted_books:
    print(book)


Книги, отсортированные по году публикации:
(1, 'The Great Gatsby', 'F. Scott Fitzgerald', 1925, 'Fiction')
(2, '1984', 'George Orwell', 1949, 'Dystopian')
(3, 'To Kill a Mockingbird', 'Harper Lee', 1960, 'Classic')


7. Ограничение количества строк

In [7]:
cursor.execute("SELECT * FROM books ORDER BY title LIMIT 2")
first_two_books = cursor.fetchall()
print("\nПервые две книги по алфавиту:")
for book in first_two_books:
    print(book)


Первые две книги по алфавиту:
(2, '1984', 'George Orwell', 1949, 'Dystopian')
(1, 'The Great Gatsby', 'F. Scott Fitzgerald', 1925, 'Fiction')


8. Агрегатные функции

In [8]:
cursor.execute("SELECT COUNT(*) FROM books")
total_books = cursor.fetchone()[0]
print(f"\nОбщее количество книг: {total_books}")


Общее количество книг: 3


9. Группировка данных

In [9]:
cursor.execute("SELECT genre, COUNT(*) FROM books GROUP BY genre")
books_by_genre = cursor.fetchall()
print("\nКоличество книг по жанрам:")
for genre, count in books_by_genre:
    print(f"{genre}: {count}")


Количество книг по жанрам:
Classic: 1
Dystopian: 1
Fiction: 1


10. Обновление данных

In [10]:
cursor.execute("UPDATE books SET year_published = 1948 WHERE title = '1984'")
conn.commit()
print("\nГод публикации книги '1984' обновлен на 1948")


Год публикации книги '1984' обновлен на 1948


11. Удаление данных


In [11]:
cursor.execute("DELETE FROM books WHERE title = 'The Great Gatsby'")
conn.commit()
print("\nКнига 'The Great Gatsby' удалена")


Книга 'The Great Gatsby' удалена


12. Создание индекса

In [12]:
cursor.execute("CREATE INDEX IF NOT EXISTS idx_author ON books(author)")
conn.commit()
print("\nИндекс для столбца author создан")


Индекс для столбца author создан


13. Проверка существования индекса

In [13]:
cursor.execute("SELECT name FROM sqlite_master WHERE type = 'index' AND name = 'idx_author'")
index_exists = cursor.fetchone()
print("\nИндекс для author существует:", bool(index_exists))


Индекс для author существует: True


14. Триггер для логирования

In [14]:

cursor.execute('''
CREATE TABLE IF NOT EXISTS logs (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    action TEXT,
    book_id INTEGER,
    timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
)
''')

cursor.execute('''
CREATE TRIGGER IF NOT EXISTS log_book_insert
AFTER INSERT ON books
BEGIN
    INSERT INTO logs (action, book_id) VALUES ('INSERT', NEW.id);
END;
''')

conn.commit()
print("\nТриггер для логирования создан")


Триггер для логирования создан


15. Транзакции

In [15]:
try:
    conn.execute("BEGIN TRANSACTION")

    cursor.execute('''
    INSERT INTO books (title, author, year_published, genre)
    VALUES ('The Catcher in the Rye', 'J.D. Salinger', 1951, 'Classic')
    ''')

    cursor.execute("UPDATE books SET year_published = 1949 WHERE title = '1984'")

    conn.commit()
    print("\nТранзакция выполнена успешно")

except sqlite3.Error as e:
    conn.rollback()
    print("\nОшибка в транзакции, изменения отменены:", e)


Транзакция выполнена успешно


16. Пользовательская функция

In [16]:
import datetime

def calculate_book_age(year_published):
    current_year = datetime.datetime.now().year
    return current_year - year_published

conn.create_function("book_age", 1, calculate_book_age)

cursor.execute("SELECT title, book_age(year_published) FROM books")
books_with_age = cursor.fetchall()
print("\nКниги с их возрастом:")
for title, age in books_with_age:
    print(f"{title}: {age} лет")


Книги с их возрастом:
1984: 76 лет
To Kill a Mockingbird: 65 лет
The Catcher in the Rye: 74 лет


17. Представления

In [17]:
cursor.execute('''
CREATE VIEW IF NOT EXISTS modern_books AS
SELECT * FROM books WHERE year_published > 1950
''')

cursor.execute("SELECT * FROM modern_books")
modern_books = cursor.fetchall()
print("\nСовременные книги (после 1950 года):")
for book in modern_books:
    print(book)


Современные книги (после 1950 года):
(3, 'To Kill a Mockingbird', 'Harper Lee', 1960, 'Classic')
(4, 'The Catcher in the Rye', 'J.D. Salinger', 1951, 'Classic')


18. Удаление таблицы

In [18]:
cursor.execute("DROP TABLE IF EXISTS books")
cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='books'")
table_exists = cursor.fetchone()
print("\nТаблица books существует после удаления:", bool(table_exists))


Таблица books существует после удаления: False


19. Композитный индекс

In [ ]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='index' AND name='idx_author_year'")
if cursor.fetchone() is None:
    cursor.execute("CREATE INDEX idx_author_year ON books(author, year_published)")
    print("\nКомпозитный индекс для author и year_published создан")
else:
    print("\nИндекс уже существует")

20. Удаление базы данных

In [20]:
import os

conn.close()

if os.path.exists('library.db'):
    os.remove('library.db')
    print("\nФайл базы данных library.db удален")
else:
    print("\nФайл базы данных library.db не существует")


Файл базы данных library.db удален


21. Обязательное задание.
Создать базу данных (БД).

In [21]:

class Student:
    def __init__(self, id, name, age, grade):
        self.id = id
        self.name = name
        self.age = age
        self.grade = grade

conn = sqlite3.connect('school.db')
cursor = conn.cursor()

cursor.execute('''
CREATE TABLE IF NOT EXISTS students (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    age INTEGER,
    grade REAL
)
''')
conn.commit()

def add_student(student):
    cursor.execute('''
    INSERT INTO students (id, name, age, grade)
    VALUES (?, ?, ?, ?)
    ''', (student.id, student.name, student.age, student.grade))
    conn.commit()


def get_all_students():
    cursor.execute("SELECT * FROM students")
    return cursor.fetchall()

student1 = Student(1, "Иван Иванов", 18, 4.5)
student2 = Student(2, "Петр Петров", 19, 3.8)

add_student(student1)
add_student(student2)

print("\nВсе студенты:")
for student in get_all_students():
    print(student)

conn.close()


Все студенты:
(1, 'Иван Иванов', 18, 4.5)
(2, 'Петр Петров', 19, 3.8)
